# **Use Case: Predictive Sales Forecasting Based on Marketing Channel Investments**

- Objective: The primary objective is to develop a machine learning model that accurately predicts future sales based on various investment levels in multiple marketing channels, including TV, Radio, Newspaper. This model will then be integrated into a Power BI dashboard, enabling stakeholders to dynamically adjust investment values for each channel and immediately view the forecasted sales outcome.

- Use Case Description: Marketing teams are continuously challenged to determine the optimal allocation of resources across digital channels to maximize sales. However, without a reliable model, it is difficult to predict how changes in marketing investment impact sales outcomes. This predictive analytics solution addresses this gap by providing a forecasted sales figure based on chosen investment levels in each channe


# **Data Loading and Preprocessing**

In [1]:
from google.colab import drive
drive.mount('/content/Drive')
file_path = '/content/Drive/MyDrive/Meine Dateien/advertising_data.csv'

Mounted at /content/Drive


In [2]:
import pandas as pd
df = pd.read_csv(file_path)
df.head(3)

,Unnamed: 0,TV,Radio,Newspaper,Sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3


In [3]:
def transform_df (df):
  df = pd.read_csv(file_path)
  df.set_index('Unnamed: 0', inplace=True)
  df.index.name = None

  df['TV'] = df['TV'] * 1000
  df['Radio'] = df['Radio'] * 1000
  df['Newspaper'] = df['Newspaper'] * 1000
  df['Sales'] = df['Sales'] * 1_000_000

  return df

df = transform_df(df)
df.head(3)

,TV,Radio,Newspaper,Sales
1,230100.0,37800.0,69200.0,22100000.0
2,44500.0,39300.0,45100.0,10400000.0
3,17200.0,45900.0,69300.0,9300000.0


In [4]:
from IPython.display import display, HTML

def data_information(df):
  describe = df.describe().round(2).T
  nan_values = df.isnull().sum().to_frame(name='NaN Values')
  duplicates = pd.DataFrame({'Duplicates': [df.duplicated().sum()]})
  rows = pd.DataFrame({'Rows': [df.shape[0]]})  # Konvertiere in DataFrame
  cols = pd.DataFrame({'Columns': [df.shape[1]]})  # Konvertiere in DataFrame

  return describe, nan_values, duplicates, rows, cols

describe, nan_values, duplicates, rows, cols = data_information(df)

describe_html = describe.to_html()
nan_values_html = nan_values.to_html()
duplicates_html = duplicates.to_html()
rows_html = rows.to_html()
cols_html = cols.to_html()

display(HTML(f"""
<div style="display: flex; gap: 20px; align-items: flex-start;">
    <div>{describe_html}</div>
    <div>{nan_values_html}</div>
    <div>{duplicates_html}</div>
    <div>{rows_html}</div>
    <div>{cols_html}</div>
</div>
"""))

In [5]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def data_information_plot(df):
  columns = list(df.columns)

  fig = make_subplots(rows=1, cols=len(columns), subplot_titles=columns)

  for i, channel in enumerate(columns):
      fig.add_trace(go.Histogram(x=df[channel], name=channel, opacity=0.75), row=1, col=i+1)


  fig.update_layout(
      title_text='Distribution of Marketing Channel Spending and Sales',
      template='plotly_white',
      height=400,
      width=1500,
      showlegend=False
    )

  fig.update_xaxes(title_text='Value')
  fig.update_yaxes(title_text='Count')

  fig.show()


data_information_plot(df)

## **Insights and Observations**
- So far we see no missing values, duplicates or anomalies which we should take care of in our data preprocessing

# **Explorative Data Analysis**




In [6]:
def channel_sales_scatter_plots(df):
    channels = [col for col in df.columns if col != 'Sales']
    fig = make_subplots(rows=1, cols=len(channels), subplot_titles=channels)

    for i, channel in enumerate(channels):
        fig.add_trace(
            go.Scatter(x=df[channel], y=df['Sales'], mode='markers', name=channel, opacity=0.7),
            row=1, col=i+1
        )

    fig.update_layout(
        title_text='Scatter Plots: Marketing Channel Investments vs. Sales',
        height=400,
        width=1800,
        template="plotly_white",
        showlegend=False
    )

    for i in range(1, len(channels) + 1):
        fig.update_xaxes(title_text='Spend', row=1, col=i)
    fig.update_yaxes(title_text='Sales', row=1, col=1)

    fig.show()

channel_sales_scatter_plots(df)

**Observations and Insights**
1. TV Spending Shows a Strong Positive Correlation with Sales:

  - As TV spending increases, there is a clear upward trend in sales, indicating a strong positive correlation. Higher investments in TV advertising tend to be associated with higher sales, suggesting that this channel may have a significant impact on driving sales growth.

2. Radio Spending Shows a Weaker Correlation with Sales:

  - Although there is a slight upward trend, the relationship between radio spending and sales is less clear and appears to be weaker than the TV channel. The scatter points are more dispersed, indicating a less consistent influence on sales.

3. Newspaper Spending Shows Minimal or No Clear Correlation with Sales:
  - The scatter plot for newspaper spending and sales appears widely dispersed, without a strong upward or downward trend. This suggests that spending on newspaper advertisements may have little to no consistent impact on sales, indicating that it might be the least effective channel among the three shown

In [7]:
def total_spending_weights(df):
  channels = [col for col in df.columns if col != 'Sales']
  total_spending = df[channels].sum()
  total_all_channels = total_spending.sum()
  spending_weights = (total_spending / total_all_channels) * 100

  result = pd.DataFrame({
      'Total Spending': total_spending,
      'Share of Total Spending (%)': spending_weights
    })

  return result

total_spending_weights(df)

,Total Spending,Share of Total Spending (%)
TV,29408500.0,73.206280
Radio,4652800.0,11.582168
Newspaper,6110800.0,15.211552


In [8]:
from sklearn.ensemble import RandomForestRegressor

def feature_importance(df):
  channels = [col for col in df.columns if col !='Sales']
  X = df[channels]
  y = df['Sales']

  model = RandomForestRegressor(n_estimators=100, random_state=42)
  model.fit(X,y)

  contributions = pd.DataFrame({
    'Channel': X.columns,
    'Contribution to Sales': model.feature_importances_
  }).sort_values(by='Contribution to Sales', ascending=False)

  return contributions, model, X, y

contributions, model, X, y = feature_importance(df)
contributions

,Channel,Contribution to Sales
0,TV,0.640357
1,Radio,0.352694
2,Newspaper,0.006949


# **Sales Prediction Using Random Forest**

In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

#Prediction on Test Dataset
y_pred = rf_model.predict(X_test)

# Evaluation
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("Root Mean Squared Error (RMSE):", rmse)
print("R² Score:", r2)

Root Mean Squared Error (RMSE): 772876.219067452
R² Score: 0.9810751065062102


In [10]:
X = df[['TV', 'Radio']]
y = df['Sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

best_model = RandomForestRegressor(n_estimators=500, random_state=42)
best_model.fit(X_train, y_train)

#Prediction on Test Dataset
y_pred = best_model.predict(X_test)

# Evaluation
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("Root Mean Squared Error (RMSE):", rmse)
print("R² Score:", r2)

Root Mean Squared Error (RMSE): 633820.6765324086
R² Score: 0.9872724046196419


In [11]:
def predict_sales(tv_spending, radio_spending, model=best_model):

    new_data = pd.DataFrame({
        'TV': [tv_spending],
        'Radio': [radio_spending]
    })

    # Prediction
    predicted_sales = model.predict(new_data)
    return predicted_sales[0]


tv_spending = 5000 # TV spending
radio_spending = 95000  # Radio spending

predicted_sales = predict_sales(tv_spending, radio_spending)
print(f"Predicted Sales for TV spending of ${tv_spending} and Radio spending of ${radio_spending}: ${predicted_sales}")


Predicted Sales for TV spending of $5000 and Radio spending of $95000: $4617000.0


In [ ]:
import joblib

# Save model
joblib.dump(best_model, 'best_model.pkl')

['best_model.pkl']

**Loading The Model From Google Drive**

In [12]:
from joblib import load

model_path = '/content/Drive/MyDrive/Meine Dateien/best_model.pkl'
test_model = load(model_path)


In [13]:
import pandas as pd

def predict_sales(tv_spending, radio_spending, model=test_model):
    new_data = pd.DataFrame({
        'TV': [tv_spending],
        'Radio': [radio_spending]
    })

    predicted_sales = model.predict(new_data)
    return predicted_sales[0]

tv_spending = 0  # TV spending
radio_spending = 0  # Radio spending

predicted_sales = predict_sales(tv_spending, radio_spending)
print(f"Predicted Sales for TV spending of ${tv_spending} and Radio spending of ${radio_spending}: ${predicted_sales}")


Predicted Sales for TV spending of $0 and Radio spending of $0: $2972600.0
